In [1]:
import numpy as np
from hmmlearn import hmm
import alfabeto_data.dissertation_images as di
from alfabeto_sources.all_sources import *
from alfabeto_code.AlfabetoConverter import transposed_pc_chords_noMMD
from Continuo.ContinuoConverter import figure_intervals_pc
import csv, copy, os
from scipy.spatial.distance import euclidean, cdist
import networkx as nx
from networkx.drawing.nx_agraph import write_dot, to_agraph
from sklearn.externals import joblib

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

ready!


In [2]:
# X = di.python_hmm(di.GetAll.all_alf, di.modal_major, 'chords')
import matplotlib as mpl
mpl.rcParams['patch.force_edgecolor'] = True
mpl.rcParams['patch.facecolor'] = 'b'

def function_graph_maker(corpus_function, corpus_name, mode, K_range, path):
    X = corpus_function
    for K in K_range:
        if not os.path.exists('%s/%s_%s_%s.pkl' % (path, corpus_name, mode, K)):
            Y = hmm.MultinomialHMM(n_components=K, verbose=True,n_iter=1000,tol=.001)
            Y.n_features=len(X[2])
            FIT_DATA = Y.fit(X[0], X[1])
            TOKEN_NAMES=X[2]
            pickleit = {'fit': FIT_DATA, 'tokens': TOKEN_NAMES}
            if not os.path.exists(path):
                os.makedirs(path)
            joblib.dump(pickleit, '%s/%s_%s_%s.pkl' %(path, corpus_name, mode, K), compress=9)
        
        FIT_DATA = joblib.load('%s/%s_%s_%s.pkl' %(path, corpus_name, mode, K))
        emiss = FIT_DATA['fit'].emissionprob_
        tokens = FIT_DATA['tokens']
        trans = FIT_DATA['fit'].transmat_

        def pie_nodes(fit_data_dict):
            emiss = fit_data_dict['fit'].emissionprob_
            tokens = fit_data_dict['tokens']
        #     K = len(emiss)
            plt.close()
            for k in range(K):
                plt.figure(figsize=(2,2), tight_layout=True)
                emit1_data = []
                emit1_labels = []
                plt.rcParams['font.size'] = 14.0
                plt.rcParams['font.family'] = 'serif'
                plt.axis('equal')
                # csfont = {'fontname':'Comic Sans MS'}
                kcolors = ['white']
                other_numb = 0
                for a, b in zip(emiss[k], tokens):
                    if a >= .03:
                        emit1_data.append(a)
                        emit1_labels.append(b)
                    else:
                        other_numb += a
                emit1_data.append(other_numb)
                emit1_labels.append('other')

                a, b = plt.pie(emit1_data, labels=emit1_labels, radius=2, colors=kcolors, labeldistance=0.8)
                for bb in b:
                    bb.set_horizontalalignment('center')
                    bb.set_fontsize(12)
                pltpath = '%s/%s/%s' %(path, corpus_name, mode)
                if not os.path.exists(pltpath):
                    os.makedirs(pltpath)
                plt.savefig(pltpath+'/%s_%s.png' %(K, k), bbox_inches='tight')
                plt.close()
        pie_nodes(FIT_DATA)

        def neato_function(K):
            import networkx as nx
            from networkx.drawing.nx_agraph import write_dot, to_agraph
            import pygraphviz as pgv
            G = nx.DiGraph()
            labels =[x+1 for x in range(K)]
            for x, y in zip(trans, labels):
                for a, b in zip(x, labels):
                    if a >= .01: #threshold*******
                        G.add_edge(y, b, penwidth=a*8, weight=int(a*100))
            edge_weights = []
            edges = G.edges(data=True)
#             print('edges:', edges)
            for x in edges:
                edge_weights.append(x[2]['penwidth'])
            nodes = G.nodes()
#             print('nodes:', nodes)
            for x in range(len(nodes)):
                G.node[nodes[x]]['shape'] = 'none'
#                 G.node[nodes[x]]['image'] = '/home/daniel/Desktop/54.svg'
                G.node[nodes[x]]['image'] = '%s/%s/%s/%s_%s.png' %(path, corpus_name, mode, K,x)
                print('%s/%s/%s/%s_%s.svg' %(path, corpus_name, mode, K,x))
                G.node[nodes[x]]['fontcolor'] = 'blue'
                G.node[nodes[x]]['fontsize'] = 40
            G = to_agraph(G)
            # G=pgv.AGraph(ranksep=10)
            if not os.path.exists(path):
                os.makedirs(path)
            print(G)
#             G.draw('tryit.pdf', format='pdf', prog='dot')
            G.draw('%s/%s_%s_%s.pdf' %(path, corpus_name, mode, K), format='pdf', prog='dot')
        neato_function(K)
        print('finished', corpus_name, mode, K)

In [3]:
# alfabeto_chords_major = di.python_hmm(di.GetAll.all_alf, di.modal_major, 'chords')
# alfabeto_chords_minor = di.python_hmm(di.GetAll.all_alf, di.modal_minor, 'chords')
alfabeto_continuo_major = di.python_hmm(di.GetAll.all_alf, di.modal_major, 'continuo')
# alfabeto_continuo_minor = di.python_hmm(di.GetAll.all_alf, di.modal_minor, 'continuo')

# palestrina_major = di.python_hmm_corpus(di.palestrina_continuo, [(0, 0), (-1, 5)])
# palestrina_minor = di.python_hmm_corpus(di.palestrina_continuo, [(0, 9), (-1, 2)])
# # josquin_major = di.python_hmm_corpus(di.josquin_continuo, [(0, 0), (-1, 5)])

# # barbershop_major = di.python_hmm_corpus(di.barbershop_continuo, di.tonal_major)

# # masses = di.python_hmm_corpus(di.zma_continuo, [(0, 0), (-1, 5)])
# # motets = di.python_hmm_corpus(di.zmo_continuo, [(0, 0), (-1, 5)])
# # songs = di.python_hmm_corpus(di.zso_continuo, di.modal_major)


# bach_major = di.python_hmm_corpus(di.bach_continuo, di.tonal_major)
# bach_minor = di.python_hmm_corpus(di.bach_continuo, di.tonal_minor)
# monteverdi_major = di.python_hmm_corpus(di.monteverdi_continuo, di.modal_major)


In [ ]:
len(masses[0])

In [4]:
# function_graph_maker(alfabeto_chords_major, 'alfabeto_chords', 'major', [11])
# function_graph_maker(alfabeto_chords_minor, 'alfabeto_chords', 'minor', [11])
function_graph_maker(alfabeto_continuo_major, 'alfabeto_continuo', 'major', [3], '/home/daniel/Desktop/hmm')
# function_graph_maker(alfabeto_continuo_minor, 'alfabeto_continuo', 'minor', [3], '/home/daniel/Desktop/hmm')

# function_graph_maker(masses, 'masses', 'major', [11], '/home/daniel/Desktop/hmm')

# function_graph_maker(alfabeto_continuo_minor, 'alfabeto_continuo', 'minor', 6)

# function_graph_maker(palestrina_major, 'palestrina', 'major', [3], '/home/daniel/Desktop/hmm')
# function_graph_maker(palestrina_minor, 'palestrina', 'minor', [3], '/home/daniel/Desktop/hmm')

# function_graph_maker(masses, 'masses', 'major', [3], '/home/daniel/Desktop/hmm')
# function_graph_maker(motets, 'motets', 'major', [3], '/home/daniel/Desktop/hmm')
# function_graph_maker(songs, 'songs', 'major', [3], '/home/daniel/Desktop/hmm')


# function_graph_maker(bach_major, 'bach', 'major', [3], '/home/daniel/Desktop/hmm')
# function_graph_maker(bach_minor, 'bach', 'minor', [3], '/home/daniel/Desktop/hmm')
# function_graph_maker(josquin_major, 'josquin', 'major', [3])
# function_graph_maker(josquin_major, 'josquin', 'major', [6])
# function_graph_maker(barbershop_major, 'barbershop', 'major', [2], 'try.pdf')
# function_graph_maker(monteverdi_major, 'monteverdi', 'major', [2], 'home/daniel/Desktop/hmm')

for f in range(2, 15):
    function_graph_maker(alfabeto_continuo_major, 'alfabeto_continuo', 'major', [f], '/home/daniel/Desktop/hmm')
    function_graph_maker(alfabeto_continuo_minor, 'alfabeto_continuo', 'minor', [f], '/home/daniel/Desktop/hmm')
    function_graph_maker(palestrina_major, 'palestrina', 'major', [f], '/home/daniel/Desktop/hmm')
    function_graph_maker(palestrina_minor, 'palestrina', 'minor', [f], '/home/daniel/Desktop/hmm')
    function_graph_maker(bach_major, 'bach', 'major', [f], '/home/daniel/Desktop/hmm')
    function_graph_maker(bach_minor, 'bach', 'minor', [f], '/home/daniel/Desktop/hmm')
# for book in range(1,8):
#     function_kaps = di.python_hmm([di.GetComposer.all_kaps[book-1]], di.modal_major, 'continuo')
#     function_graph_maker(function_kaps, 'kaps%s' % book, 'major', [4])
#     function_graph_maker(function_kaps, 'kaps%s' % book, 'major', [5])

         1      -34748.1495             +nan
         2      -25253.3814       +9494.7681
         3      -25225.5483         +27.8332
         4      -25200.5562         +24.9921
         5      -25172.5386         +28.0175
         6      -25136.7330         +35.8056
         7      -25089.0476         +47.6854
         8      -25026.5828         +62.4648
         9      -24948.7796         +77.8032
        10      -24858.8000         +89.9797
        11      -24763.7572         +95.0427
        12      -24671.5849         +92.1723
        13      -24585.3927         +86.1922
        14      -24500.3224         +85.0703
        15      -24405.8274         +94.4949
        16      -24292.0554        +113.7720
        17      -24159.5908        +132.4647
        18      -24026.6878        +132.9030
        19      -23916.0603        +110.6275
        20      -23833.8566         +82.2037
        21      -23772.4909         +61.3657
        22      -23722.7650         +49.7260
        23

/home/daniel/Desktop/hmm/alfabeto_continuo/major/3_0.svg
/home/daniel/Desktop/hmm/alfabeto_continuo/major/3_1.svg
/home/daniel/Desktop/hmm/alfabeto_continuo/major/3_2.svg
digraph "" {
	1	 [fontcolor=blue,
		fontsize=40,
		image="/home/daniel/Desktop/hmm/alfabeto_continuo/major/3_0.png",
		shape=none];
	1 -> 1	 [penwidth=0.66566841183,
		weight=8];
	2	 [fontcolor=blue,
		fontsize=40,
		image="/home/daniel/Desktop/hmm/alfabeto_continuo/major/3_1.png",
		shape=none];
	1 -> 2	 [penwidth=3.06278411517,
		weight=38];
	3	 [fontcolor=blue,
		fontsize=40,
		image="/home/daniel/Desktop/hmm/alfabeto_continuo/major/3_2.png",
		shape=none];
	1 -> 3	 [penwidth=4.271547473,
		weight=53];
	2 -> 1	 [penwidth=4.98357699193,
		weight=62];
	2 -> 2	 [penwidth=0.827753217724,
		weight=10];
	2 -> 3	 [penwidth=2.18866979035,
		weight=27];
	3 -> 1	 [penwidth=1.8344658323,
		weight=22];
	3 -> 2	 [penwidth=5.82355656268,
		weight=72];
	3 -> 3	 [penwidth=0.341977605023,
		weight=4];
}

finished alfabeto_continuo 

         1      -33938.9900             +nan
         2      -25258.1219       +8680.8681
         3      -25235.9160         +22.2059
         4      -25219.9188         +15.9972
         5      -25206.5470         +13.3718
         6      -25193.7784         +12.7686
         7      -25180.2776         +13.5008
         8      -25164.9520         +15.3256
         9      -25146.7174         +18.2346
        10      -25124.4017         +22.3157
        11      -25096.7724         +27.6293
        12      -25062.7235         +34.0489
        13      -25021.6724         +41.0512
        14      -24974.1462         +47.5262
        15      -24922.2864         +51.8598
        16      -24869.6430         +52.6434
        17      -24819.8360         +49.8070
        18      -24774.9591         +44.8769
        19      -24735.2569         +39.7022
        20      -24700.0454         +35.2114
        21      -24668.5987         +31.4467
        22      -24640.4164         +28.1823
        23

/home/daniel/Desktop/hmm/alfabeto_continuo/major/2_0.svg
/home/daniel/Desktop/hmm/alfabeto_continuo/major/2_1.svg
digraph "" {
	1	 [fontcolor=blue,
		fontsize=40,
		image="/home/daniel/Desktop/hmm/alfabeto_continuo/major/2_0.png",
		shape=none];
	1 -> 1	 [penwidth=3.6242702039,
		weight=45];
	2	 [fontcolor=blue,
		fontsize=40,
		image="/home/daniel/Desktop/hmm/alfabeto_continuo/major/2_1.png",
		shape=none];
	1 -> 2	 [penwidth=4.3757297961,
		weight=54];
	2 -> 1	 [penwidth=7.27486943048,
		weight=90];
	2 -> 2	 [penwidth=0.72513056952,
		weight=9];
}

finished alfabeto_continuo major 2


NameError: name 'alfabeto_continuo_minor' is not defined

In [ ]:
all_scores = []
for w in range(2, 15):
    for x in range(500):
        a = joblib.load('/home/daniel/Desktop/hmm/alfabeto_continuo/major/%s/%s.pkl' %(w,x))
        all_scores.append(((w,x),a['fit'].monitor_.converged))

In [ ]:
for x in all_scores:
    if x[-1] == False:
        print(x)

In [ ]:
max(all_scores)

In [ ]:
hmmm = [[1],[3],[2],[11],[2],[1],[6],[9]]

In [ ]:
a = joblib.load('/home/daniel/Desktop/hmm/alfabeto_continuo/major/3/0.pkl')

In [ ]:
a['fit'].monitor_.converged

In [ ]:
os.path.exists('C://Users/Daniel/Desktop/')

In [ ]:
di.function_mapper(di.GetAll.all_alf, di.modal_major, di.alfabeto_continuo_major, 'alfaeto continuo', '/home/daniel/Desktop/alf.pdf')

In [4]:
"""Parallel Processing"""

import ipyparallel as ipp
from sklearn.externals import joblib
import os
import numpy as np

rc = ipp.Client()

print(rc.ids)
dview = rc[:]
dview.execute('from sklearn.externals import joblib')
dview.execute('import os')
dview.execute('import numpy as np')
dview.execute('import alfabeto_data.dissertation_images as di')
dview.execute('palestrina_major = di.python_hmm_corpus(di.palestrina_continuo, [(0, 0), (-1, 5)])')
dview.execute('palestrina_minor = di.python_hmm_corpus(di.palestrina_continuo, [(0, 9), (-1, 2)])')
dview.execute('bach_major = di.python_hmm_corpus(di.bach_continuo, di.tonal_major)')
dview.execute('bach_minor = di.python_hmm_corpus(di.bach_continuo, di.tonal_minor)')
dview.execute('alfabeto_continuo_major = di.python_hmm(di.GetAll.all_alf, di.modal_major, "continuo")')
dview.execute('alfabeto_continuo_minor = di.python_hmm(di.GetAll.all_alf, di.modal_minor, "continuo")')
dview.execute('import matplotlib.pyplot as plt')
dview.execute('from hmmlearn import hmm')
dview.execute('import networkx as nx')
dview.execute('from networkx.drawing.nx_agraph import write_dot, to_agraph')


[0, 1, 2, 3, 4, 5, 6, 7]


<AsyncResult: execute>

In [7]:
@dview.parallel(block=True)
def function_graph_parallel(dict_number):
    parallel_dict = {}
    thing_number = 0
    corpus_dict = {0:{'name': 'palestrina', 'mode': 'major', 'corpus': palestrina_major}, 
                   1:{'name': 'palestrina', 'mode': 'minor', 'corpus': palestrina_minor}, 
                   2:{'name': 'bach', 'mode': 'major', 'corpus': bach_major}, 
                   3:{'name': 'bach', 'mode': 'minor', 'corpus': bach_minor},
                   4:{'name': 'alfabeto_continuo', 'mode': 'major', 'corpus': alfabeto_continuo_major}, 
                   5:{'name': 'alfabeto_continuo', 'mode': 'minor', 'corpus': alfabeto_continuo_minor}}
    for x in range(len(corpus_dict)):
        for k in range(2, 16):
            parallel_dict[thing_number] = {'data':corpus_dict[x], 'K_range': [k]}
            thing_number += 1
    corpus_function = parallel_dict[dict_number]['data']['corpus']
    corpus_name = parallel_dict[dict_number]['data']['name']
    mode = parallel_dict[dict_number]['data']['mode']
    K_range = parallel_dict[dict_number]['K_range']
    path = '/home/daniel/Desktop/hmm'
    
    def function_graph_maker(corpus_function, corpus_name, mode, K_range, path):
        X = corpus_function
        for K in K_range:
            if not os.path.exists('%s/%s_%s_%s.pkl' % (path, corpus_name, mode, K)):
                Y = hmm.MultinomialHMM(n_components=K, verbose=True,n_iter=1000,tol=.001)
                Y.n_features=len(X[2])
                FIT_DATA = Y.fit(X[0], X[1])
                TOKEN_NAMES=X[2]
                pickleit = {'fit': FIT_DATA, 'tokens': TOKEN_NAMES}
                if not os.path.exists(path):
                    os.makedirs(path)
                joblib.dump(pickleit, '%s/%s_%s_%s.pkl' %(path, corpus_name, mode, K), compress=9)

            FIT_DATA = joblib.load('%s/%s_%s_%s.pkl' %(path, corpus_name, mode, K))
            emiss = FIT_DATA['fit'].emissionprob_
            tokens = FIT_DATA['tokens']
            trans = FIT_DATA['fit'].transmat_

            def pie_nodes(fit_data_dict):
                emiss = fit_data_dict['fit'].emissionprob_
                tokens = fit_data_dict['tokens']
            #     K = len(emiss)
                plt.close()
                for k in range(K):
                    plt.figure(figsize=(2,2), tight_layout=True)
                    emit1_data = []
                    emit1_labels = []
                    plt.rcParams['font.size'] = 14.0
                    plt.rcParams['font.family'] = 'serif'
                    plt.axis('equal')
                    # csfont = {'fontname':'Comic Sans MS'}
                    kcolors = ['white']
                    other_numb = 0
                    for a, b in zip(emiss[k], tokens):
                        if a >= .03:
                            emit1_data.append(a)
                            emit1_labels.append(b)
                        else:
                            other_numb += a
                    emit1_data.append(other_numb)
                    emit1_labels.append('other')

                    a, b = plt.pie(emit1_data, labels=emit1_labels, radius=2, colors=kcolors, labeldistance=0.8)
                    for bb in b:
                        bb.set_horizontalalignment('center')
                        bb.set_fontsize(12)
                    pltpath = '%s/%s/%s' %(path, corpus_name, mode)
                    if not os.path.exists(pltpath):
                        os.makedirs(pltpath)
                    plt.savefig(pltpath+'/%s_%s.png' %(K, k), bbox_inches='tight')
                    plt.close()
            pie_nodes(FIT_DATA)

            def neato_function(K):
                import networkx as nx
                from networkx.drawing.nx_agraph import write_dot, to_agraph
                import pygraphviz as pgv
                G = nx.DiGraph()
                labels =[x+1 for x in range(K)]
                for x, y in zip(trans, labels):
                    for a, b in zip(x, labels):
                        if a >= .1: #threshold*******
                            G.add_edge(y, b, penwidth=a*8, weight=int(a*100))
                edge_weights = []
                edges = G.edges(data=True)
    #             print('edges:', edges)
                for x in edges:
                    edge_weights.append(x[2]['penwidth'])
                nodes = G.nodes()
    #             print('nodes:', nodes)
                for x in range(len(nodes)):
                    G.node[nodes[x]]['shape'] = 'none'
    #                 G.node[nodes[x]]['image'] = '/home/daniel/Desktop/54.svg'
                    G.node[nodes[x]]['image'] = '%s/%s/%s/%s_%s.png' %(path, corpus_name, mode, K,x)
                    print('%s/%s/%s/%s_%s.svg' %(path, corpus_name, mode, K,x))
                    G.node[nodes[x]]['fontcolor'] = 'blue'
                    G.node[nodes[x]]['fontsize'] = 40
                G = to_agraph(G)
                # G=pgv.AGraph(ranksep=10)
                if not os.path.exists(path):
                    os.makedirs(path)
                print(G)
    #             G.draw('tryit.pdf', format='pdf', prog='dot')
                G.draw('%s/%s_%s_%s.pdf' %(path, corpus_name, mode, K), format='pdf', prog='dot')
            neato_function(K)
            print('finished', corpus_name, mode, K)
    function_graph_maker(corpus_function, corpus_name, mode, K_range, path)
    return corpus_name, mode, K_range

In [8]:
function_graph_parallel.map([k for k in range(84)])

[('palestrina', 'major', [2]),
 ('palestrina', 'major', [3]),
 ('palestrina', 'major', [4]),
 ('palestrina', 'major', [5]),
 ('palestrina', 'major', [6]),
 ('palestrina', 'major', [7]),
 ('palestrina', 'major', [8]),
 ('palestrina', 'major', [9]),
 ('palestrina', 'major', [10]),
 ('palestrina', 'major', [11]),
 ('palestrina', 'major', [12]),
 ('palestrina', 'major', [13]),
 ('palestrina', 'major', [14]),
 ('palestrina', 'major', [15]),
 ('palestrina', 'minor', [2]),
 ('palestrina', 'minor', [3]),
 ('palestrina', 'minor', [4]),
 ('palestrina', 'minor', [5]),
 ('palestrina', 'minor', [6]),
 ('palestrina', 'minor', [7]),
 ('palestrina', 'minor', [8]),
 ('palestrina', 'minor', [9]),
 ('palestrina', 'minor', [10]),
 ('palestrina', 'minor', [11]),
 ('palestrina', 'minor', [12]),
 ('palestrina', 'minor', [13]),
 ('palestrina', 'minor', [14]),
 ('palestrina', 'minor', [15]),
 ('bach', 'major', [2]),
 ('bach', 'major', [3]),
 ('bach', 'major', [4]),
 ('bach', 'major', [5]),
 ('bach', 'major', [

In [ ]:
parallel_dict = {}
thing_number = 0
corpus_dict = {0:{'name': 'palestrina', 'mode': 'major', 'corpus': palestrina_major}, 
               1:{'name': 'palestrina', 'mode': 'minor', 'corpus': palestrina_minor}, 
               2:{'name': 'bach', 'mode': 'major', 'corpus': bach_major}, 
               3:{'name': 'bach', 'mode': 'minor', 'corpus': bach_minor},
               4:{'name': 'alfabeto_continuo', 'mode': 'major', 'corpus': alfabeto_continuo_major}, 
               5:{'name': 'alfabeto_continuo', 'mode': 'minor', 'corpus': alfabeto_continuo_minor}}
for x in range(len(corpus_dict)):
    for k in range(2, 15):
        parallel_dict[thing_number] = {'data':corpus_dict[x], 'K_range': [k]}
        thing_number += 1

In [ ]:
thing_number = 0
a = [0, 1, 2, 3, 4, 5]
for x in a:
    for y in range(2, 15)

In [ ]:
parallel_dict[0]['data']['mode']

In [ ]:
14*6
